# Library 정보 수집 

In [ ]:
import requests
from enum import Enum
import pandas as pd
from pandas.io.json import json_normalize
import xmltodict
import json
import cx_Oracle
from datetime import datetime
import schedule
import time

class LibCode(Enum):
    Code01 = 124004
    Code02 = 124012
    Code03 = 129224
    Code04 = 129219
    Code05 = 124007
    

def initDB():
    # 최신데이터로 업데이트하기 위하여 기존 데이터 삭제
    connect = cx_Oracle.connect("Library_admin", "12345", "localhost:1521/xe")
    cursor = connect.cursor()
    sql_delete = 'delete from book'
    cursor.execute(sql_delete)
    cursor.close()
    connect.commit()
    connect.close()
    print('[' + datetime.now().strftime('%Y%m%d') + ']','DB 초기화 완료.....')
    
def collectionBook():
    oriUrl = 'http://data4library.kr/api/loanItemSrchByLib'
    myKey = 'af2df55209c61c0726e45405de83247e14d2474c9c0bca948854d5495b6465a1'
    region = '24'  #광주
    dtl_region = '24030'  #광주 남구
    pageSize = '10'
    count = 0
    checkDB = False
    for code in LibCode:
        url = oriUrl + "?authKey=" + myKey + '&startDt=2022-01-01&endDt=2022-11-21' + \
            '&pageNo=1&pageSize=' + pageSize + "&libCode=" + str(code.value)
        response = requests.get(url)
        
        # XML 수집
        content = response.text
        
        # XML -> JSON 변환
        result = xmltodict.parse(content)
        dictionay = json.loads(json.dumps(result))
        if dictionay != "" and checkDB == False:
            initDB()
            checkDB = True
        elif checkDB == True:
            pass
        else:
            print('[' + datetime.now().strftime('%Y%m%d') + ']','데이터 수집 오류 발생.....')
            return
        connect = cx_Oracle.connect("Library_admin", "12345", "localhost:1521/xe")
        cursor = connect.cursor()
        df=json_normalize(dictionay['response']['docs']['doc'])
        columns = ['bookname','authors','isbn13','bookImageURL']
        df = df[columns]
        for i in df.index:
            sql_insert = 'INSERT INTO BOOK(book_number,bookname,authors,isbn13,book_ImageURL)' +\
            'VALUES(:booknumber, :bookname, :authors, :isbn13, :bookImageURL)'
            
            booknumber = count
            bookname = df.loc[i].bookname
            authors = df.loc[i].authors
            isbn13 = df.loc[i].isbn13
            bookImageURL = df.loc[i].bookImageURL
            
            cursor.execute(sql_insert, (booknumber, bookname, authors, isbn13, bookImageURL))
            connect.commit()
            count += 1
        connect.close()
    
    print('[' + datetime.now().strftime('%Y%m%d') + ']','DB 데이터 입력 완료.....')
    

schedule.every().day.at("12:25:30").do(collectionBook)

while True:
    schedule.run_pending()
    time.sleep(1)

[20221124] DB 초기화 완료.....


C:\Users\big303\AppData\Local\Temp\ipykernel_11748\635763160.py:60: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df=json_normalize(dictionay['response']['docs']['doc'])
C:\Users\big303\AppData\Local\Temp\ipykernel_11748\635763160.py:60: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df=json_normalize(dictionay['response']['docs']['doc'])
C:\Users\big303\AppData\Local\Temp\ipykernel_11748\635763160.py:60: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df=json_normalize(dictionay['response']['docs']['doc'])
C:\Users\big303\AppData\Local\Temp\ipykernel_11748\635763160.py:60: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df=json_normalize(dictionay['response']['docs']['doc'])
C:\Users\big303\AppData\Local\Temp\ipykernel_11748\635763160.py:60: FutureWarning: pandas.io.json.json_normalize is 

[20221124] DB 데이터 입력 완료.....
